In [1]:
import pandas as pd
import torch
from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import QuantileLoss
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


c:\Users\trh00\miniconda3\envs\TFT_env\lib\site-packages\pytorch_forecasting\models\base_model.py:30: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [11]:
traffic_data = pd.read_csv('dataset1.csv')

In [13]:
filtered_data = traffic_data[traffic_data['trafficID'].isin(['C1', 'C2', 'C3', 'C4'])]['num_cars']
# for row in filtered_data.itertuples():
#     print(row)
print(max(filtered_data))
traffic_data['DateTime'] = pd.to_datetime(traffic_data['timeStamp'])

# Calculate time index in seconds from the first timestamp
traffic_data['time_idx'] = (traffic_data['DateTime'] - traffic_data['DateTime'].iloc[0]).dt.total_seconds()
traffic_data['time_idx'] = traffic_data['time_idx'].astype(int)
print(traffic_data['time_idx'])

101
0          0
1          0
2          0
3          0
4          5
        ... 
1425    4651
1426    4651
1427    4651
1428    4656
1429    4656
Name: time_idx, Length: 1430, dtype: int32


C:\Users\trh00\AppData\Local\Temp\ipykernel_12200\3950167389.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  traffic_data['DateTime'] = pd.to_datetime(traffic_data['timeStamp'])


In [4]:
filtered_data = traffic_data[traffic_data['trafficID'].isin(['A1', 'A2', 'A3', 'A4'])]['num_cars']
# for row in filtered_data.itertuples():
#     print(row)
print(max(filtered_data))


119


In [19]:
# Load and preprocess your data
# Assuming 'traffic_data' is your DataFrame after preprocessing
encoder = OneHotEncoder(sparse=False)
scaler = MinMaxScaler()

traffic_data['num_cars'] = scaler.fit_transform(traffic_data[['num_cars']])
# Assuming 'traffic_data' is your DataFrame and 'trafficID' is a column in it.
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Apply OneHotEncoder to the 'trafficID' column
encoded_trafficID = encoder.fit_transform(traffic_data[['trafficID']])

# Create new column names for the encoded features
encoded_columns = [f'trafficID_encoded_{i}' for i in range(encoded_trafficID.shape[1])]

# Add the encoded features as new columns to the DataFrame
traffic_data[encoded_columns] = encoded_trafficID

# Train-validation split
train, val = train_test_split(traffic_data, test_size=0.2, random_state=42)

# Define parameters for the dataset and dataloader
max_encoder_length = 300
max_prediction_length = 10

training = TimeSeriesDataSet(
    train,
    time_idx='time_idx',  # This should be a numeric index of time
    target='greenTime',
    group_ids=['trafficID'],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=['trafficID'],
    time_varying_known_reals=['encoded_trafficID', 'weatherStamp', 'num_cars'],
    time_varying_unknown_reals=['num_cars'],
    scalers={'greenTime': GroupNormalizer(groups=['trafficID'], transformation='softplus')}
)

validation = TimeSeriesDataSet.from_dataset(training, val, predict=True)
batch_size = 16
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=4)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=4)

# Define the Temporal Fusion Transformer Model
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=1,
    loss=QuantileLoss()
)

# Train the model
trainer = torch.optim.Adam(tft.parameters(), lr=0.01)
epochs = 20

for epoch in range(epochs):
    tft.train()
    for batch in train_dataloader:
        loss = tft.training_step(batch, batch_idx=0)
        loss.backward()
        trainer.step()
        trainer.zero_grad()

    with torch.no_grad():
        tft.eval()
        val_loss = sum(tft.validation_step(batch, batch_idx=0)['loss'] for batch in val_dataloader) / len(val_dataloader)
    print(f'Epoch {epoch}, Validation Loss: {val_loss}')

# Optionally, save the model
torch.save(tft.state_dict(), 'tft_model.pth')


c:\Users\trh00\miniconda3\envs\TFT_env\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


KeyError: "None of [Index(['encoded_trafficID'], dtype='object')] are in the [columns]"